# Categorize Accidents Using Sentence Transformer and Linear learner

![Workflow](./inference-pipeline.drawio.png)

In this notebook, we will demonstrate categorizing accidents using sentence transformer and a simple classifier. 

First, we fine tune pretrained `bert-base-uncased` model from `HuggingFace Library` in an unsupervised fashion, on `Industrial labor accident data`. The objective is to find the similar accident reports based on the description of the incident using `bert-base-uncased`. 

Second, we train an linear learner classification model using incident features and similar incidents' features.

At the end, we deploy an inference pipeline which takes an incident report as input and predict the accident type.

## Setup
Update sagemaker package and restart the kernel. 

In [16]:
!pip install -U sagemaker -q
# !pip install sentence_transformers -q
# !pip install ipywidgets

In [17]:
import sagemaker
sagemaker.__version__

'2.217.0'

In [6]:
import boto3, os, sagemaker
import json

sess = sagemaker.Session()
bucket = sess.default_bucket() 
prefix = 'sentencetransformer/input'
role = sagemaker.get_execution_role()

## Dataset

Download the dataset from: https://www.kaggle.com/ihmstefanini/industrial-safety-and-health-analytics-database and upload the downloaded csv file to the notebook. 

The database is basically records of accidents from 12 different plants in 03 different countries which every line in the data is an occurrence of an accident.

**Columns description**
- Data: timestamp or time/date information
- Countries: which country the accident occurred (anonymized)
- Local: the city where the manufacturing plant is located (anonymized)
- Industry sector: which sector the plant belongs to
- Accident level: from I to VI, it registers how severe was the accident (I means not severe but VI means very severe)
- Potential Accident Level: Depending on the Accident Level, the database also registers how severe the accident could have been (due to other factors involved in the accident)
- Genre: if the person is male of female
- Employee or Third Party: if the injured person is an employee or a third party
- Critical Risk: some description of the risk involved in the accident
- Description: Detailed description of how the accident happened.

In [ ]:
import pandas as pd
df_data = pd.read_csv('IHMStefanini_industrial_safety_and_health_database_with_accidents_description.csv', index_col=0)

In [ ]:
df_data.head()

In [ ]:
df_data['Industry Sector'].hist()

In [ ]:
df_data['Genre'].hist()

### Upload data to s3

In [7]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train.csv')).upload_file('IHMStefanini_industrial_safety_and_health_database_with_accidents_description.csv')
training_input_path = "s3://{}/{}/train.csv".format(bucket,prefix)
training_input_path

's3://sagemaker-us-east-1-827930657850/sentencetransformer/input/train.csv'

## Fine Tuning Sentence Transformer on your Dataset

### Setting hyper-parameters

In [8]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 8,
                 'model_name':'bert-base-uncased'
                 }

The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, including the following:

- **SM_MODEL_DIR**: A string that represents the path where the training job writes the model artifacts to. After training, artifacts in this directory are uploaded to S3 for model hosting. SM_MODEL_DIR is always set to /opt/ml/model.

- **SM_NUM_GPUS**: An integer representing the number of GPUs available to the host.

- **SM_CHANNEL_XXXX**: A string that represents the path to the directory that contains the input data for the specified channel. For example, if you specify two input channels in the HuggingFace estimator’s fit call, named train and test, the environment variables SM_CHANNEL_TRAIN and SM_CHANNEL_TEST are set.

You can find a full list of the exposed environment variables [here](#https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md).

Later we define hyperparameters in the HuggingFace Estimator, which are passed in as named arguments and and can be processed with the [ArgumentParser()](#https://huggingface.co/docs/sagemaker/train#create-an-huggingface-estimator).

In [ ]:
!pygmentize ./code/unsupervised.py

In [9]:
huggingface_estimator = HuggingFace(entry_point='unsupervised.py',
                            source_dir='./code',
                            instance_type='ml.p3.2xlarge', # GPU supported by Hugging Face
                            instance_count=1,
                            role=role,
                            transformers_version='4.6',
                            pytorch_version='1.7',
                            py_version='py36',
                            hyperparameters = hyperparameters)

In [10]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2024-05-14-15-24-39-469


2024-05-14 15:24:40 Starting - Starting the training job...
2024-05-14 15:25:07 Pending - Training job waiting for capacity...
2024-05-14 15:25:31 Pending - Preparing the instances for training...
2024-05-14 15:26:06 Downloading - Downloading input data...
2024-05-14 15:26:17 Downloading - Downloading the training image...............
2024-05-14 15:28:58 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-05-14 15:29:26,776 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-05-14 15:29:26,805 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-05-14 15:29:26,808 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-05-14 15:29:26,991 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 

In [11]:
huggingface_estimator.model_data

's3://sagemaker-us-east-1-827930657850/huggingface-pytorch-training-2024-05-14-15-24-39-469/output/model.tar.gz'

## Deploy Fine-tuned Sentence Transformer

We will deploy the `Sentence Transformer` model using `SageMaker HuggingFaceModel` object with `inference.py` script as an entrypoint. 

Let's take a look into the `inference` script which is in the `code` directory and add the bucket name where you have the training data. Also, don't forget to update the `s3key`. This data will act as the source data, against which we will compare our target sentence. In this case, based on the description of the incident, model will find the similar accident reports.  

In [ ]:
!pygmentize ./code/inference.py

In [12]:
from sagemaker.huggingface.model import HuggingFaceModel
sentence_transformer = HuggingFaceModel(model_data = huggingface_estimator.model_data, 
                                    role = role, 
                                    source_dir = 'code',
                                    entry_point = 'inference.py', 
                                    transformers_version='4.6',
                                    pytorch_version='1.7',
                                    py_version='py36',)

## Deploy endpoint and test

In [ ]:
predictor = sentence_transformer.deploy(initial_instance_count = 1, instance_type = 'ml.g4dn.2xlarge')

In [ ]:
prediction = predictor.predict("they saw the bee carton, the reaction was to move away from the box as quickly as possible to avoid the stings, they ran about 50 meters, looking for a safe area, to exit the radius of attack of the bees, but the S.S. and Breno), were attacked and consequently they suffered 02 stings, in the belly and Jehovah in the hand, verified that there was no type of allergic reaction, returned with the normal activities.")

In [ ]:
# the returned prediction is in json format, can add output_fn function in inference script to covert it to csv format
result = json.loads(prediction)
result = result['result']
result

In [ ]:
sm_model.deploy(initial_instance_count=1, instance_type="ml.g4dn.2xlarge", endpoint_name=endpoint_name)

In [ ]:
sm_client = sess.boto_session.client("sagemaker")
sm_client.delete_endpoint(EndpointName='huggingface-pytorch-inference-2024-04-10-17-48-52-730')

## Testing with batch transform

In [ ]:
testing_input_path = "s3://{}/{}/incident-batch.jsonl".format(bucket,prefix)

In [45]:
testing_output_path = "s3://{}/{}".format(bucket,prefix)

In [46]:
testing_output_path

's3://sagemaker-us-east-1-827930657850/sentencetransformer/input'

In [ ]:
batch_job = sentence_transformer.transformer(
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    output_path=testing_output_path,
    strategy='SingleRecord')


batch_job.transform(
    data=testing_input_path,
    content_type='application/json',    
    split_type='Line')

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "1.2-1"
script_path = "feature-processing.py"

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.m5.xlarge",
    sagemaker_session=sess,
)

In [ ]:
sklearn_preprocessor.fit({"train": 's3://sagemaker-us-east-1-827930657850/sentencetransformer/input'})

In [ ]:
train_input = 's3://sagemaker-us-east-1-827930657850/sentencetransformer/input/train.csv'
transformer = sklearn_preprocessor.transformer(
    instance_count=1, instance_type="ml.m5.xlarge", assemble_with="Line", accept="text/csv"
)
# Preprocess training input
transformer.transform(train_input, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

In [ ]:
preprocessed_train

In [ ]:
from sagemaker.image_uris import retrieve

ll_image = retrieve("linear-learner", boto3.Session().region_name)
s3_ll_output_key_prefix = "ll_training_output"
s3_ll_output_location = "s3://{}/{}/{}/{}".format(
    bucket, prefix, s3_ll_output_key_prefix, "ll_model"
)

ll_estimator = sagemaker.estimator.Estimator(
    ll_image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=20,
    max_run=3600,
    input_mode="File",
    output_path=s3_ll_output_location,
    sagemaker_session=sess,
)

ll_estimator.set_hyperparameters(feature_dim=4, predictor_type="regressor", mini_batch_size=1)

ll_train_data = sagemaker.inputs.TrainingInput(
    preprocessed_train,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)

data_channels = {"train": ll_train_data}
ll_estimator.fit(inputs=data_channels, logs=True)

In [ ]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inference_model = sklearn_preprocessor.create_model()
linear_learner_model = ll_estimator.create_model()

model_name = "inference-pipeline-" + timestamp_prefix
endpoint_name = "inference-pipeline-ep-" + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, role=role, models=[sentence_transformer, scikit_learn_inference_model]#, linear_learner_model]
)

sm_model.deploy(initial_instance_count=1, instance_type='ml.g4dn.2xlarge', endpoint_name=endpoint_name)

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

payload = "they saw the bee carton, the reaction was to move away from the box as quickly as possible to avoid the stings, they ran about 50 meters, looking for a safe area, to exit the radius of attack of the bees, but the S.S. and Breno), were attacked and consequently they suffered 02 stings, in the belly and Jehovah in the hand, verified that there was no type of allergic reaction, returned with the normal activities."

predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sess, serializer=CSVSerializer()
)

print(predictor.predict(payload))

## Model Building Pipeline

In [72]:
!python pipeline.py 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /root/sagemaker-inference-pipeline-byos
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2024-05-15-21-42-24-718
2024-05-15 21:42:26 Starting - Starting the training job...
2024-05-15 21:42:27 Pending - Training job waiting for capacity..................
2024-05-15 21:45:41 Pending - Preparing the instances for training...
2024-05-15 21:46:18 Downloading - Downloading input data...
2024-05-15 21:46:38 Downloading - Downloading the training image...............
2024-05-15 21:49:09 Training - Training image download completed. Training in progress...bash: cannot set terminal proce

## (Optional) Alternative way of processing features

In [ ]:
train_path = f"s3://{bucket}/{prefix}/train"
validation_path = f"s3://{bucket}/{prefix}/validation"
test_path = f"s3://{bucket}/{prefix}/test"

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role


sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=get_execution_role(),
    instance_type="ml.m5.large",
    instance_count=1, 
    base_job_name='newsela-skprocessing'
)

# run as a processing job
sklearn_processor.run(
    code='preprocessing.py',
    inputs=[
        ProcessingInput(
            source=training_input_path, 
            destination="/opt/ml/processing/input/report",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key"
        ),
        ProcessingInput(
            source=similar_report_path, 
            destination="/opt/ml/processing/input/similar_report",
            s3_input_mode="File",
            s3_data_distribution_type="ShardedByS3Key"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train_data", 
            source="/opt/ml/processing/output/train",
            destination=train_path,
        ),
        ProcessingOutput(output_name="validation_data", source="/opt/ml/processing/output/validation", destination=validation_path),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/output/test", destination=test_path),
    ]
)